In [ ]:
# This is the routine that i used to convert the csvs of all years one by one

# 1. Read Dataframe from csv and specify datatypes

# 2. Optimize dataypes for memory usage

# 3. Fill all missing values in all columns with nanplaceholder
# This seemed necessary in order to be able to compare rows to count and remove duplicates. I encountered
# problems before: the comparison didnt work because NaN != NaN

# 4. Count number of identical occurences and write in new column, then remove dupes

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

# set option so that e.g. df.head() will show all columns and none are omitted 
pd.set_option('display.max_columns', None)

import os
print("Current working directory: ", os.getcwd())

# Specify the year
year = "2022"  # Change this to the desired year

# Define file paths based on the year
csv_file_path = f"..\\Data\\EU\\EU {year}.csv"
parquet_file_path = f"..\\Data\\EU\\EU {year}_dtypes.parquet"
parquet_nanplaceholder_path = f"..\\Data\\EU\\EU {year}_dtypes_nanplaceholder.parquet"
parquet_dupecounts_reduced_path = f"..\\Data\\EU\\EU {year}_dtypes_nanplaceholder_dupecounts_reduced.parquet"

# 1. Read Dataframe from csv and specify datatypes. "category" for all string values reduces memory usage a lot.
df = pd.read_csv(csv_file_path, dtype={
    'ID': 'Int32',
    'Country': 'category',
    'VFN': 'category',
    'Mp': 'category',
    'Mh': 'category',
    'Man': 'category',
    'MMS': 'category',
    'Tan': 'category',
    'T': 'category',
    'Va': 'category',
    'Ve': 'category',
    'Mk': 'category',
    'Cn': 'category',
    'Ct': 'category',
    'Cr': 'category',
    'r': 'Int32',
    'm (kg)': 'Int32',
    'Mt': 'Int32',
    'Enedc (g/km)': 'Int32',
    'Ewltp (g/km)': 'Int32',
    'W (mm)': 'Int32',
    'At1 (mm)': 'Int32',
    'At2 (mm)': 'Int32',
    'Ft': 'category',
    'Fm': 'category',
    'ec (cm3)': 'Int32',
    'ep (KW)': 'Int32',
    'z (Wh/km)': 'Int32',
    'IT': 'category',
    'Ernedc (g/km)': 'Float64',
    'Erwltp (g/km)': 'Float64',
    'De': 'Float64',
    'Vf': 'Int32',
    'Status': 'category',
    'year': 'Int32',
    'Date of registration': 'category',
    'Fuel consumption ': 'Float64',
    'ech': 'category',
    'RLFI': 'category',
    'Electric range (km)': 'Int32',
})

print(f"Dataframe for year {year} has been read from CSV and datatypes specified:")
print(df.info())

#----------------------------------------------------------------------------------
# 2. Optimize dataypes for memory usage

def optimize_dtypes(df):
    # Calculate total memory usage before optimization
    before_mem = df.memory_usage(deep=True).sum()
    print(f"Total memory usage before optimization: {before_mem / 1024**2:.2f} MB")
    
    # Downcast integer columns in place
    int_cols = df.select_dtypes(include='int').columns
    df[int_cols] = df[int_cols].apply(pd.to_numeric, downcast='integer')
    
    # Downcast float columns in place
    float_cols = df.select_dtypes(include='float').columns
    df[float_cols] = df[float_cols].apply(pd.to_numeric, downcast='float')
    
    # Calculate total memory usage after optimization
    after_mem = df.memory_usage(deep=True).sum()
    print(f"Total memory usage after optimization: {after_mem / 1024**2:.2f} MB")
    
optimize_dtypes(df)

# Save to parquet format (is able to store categorized data in its reduced format)
df.to_parquet(parquet_file_path)
print(f"Dataframe with optimized datatypes has been saved as parquet to {parquet_file_path}")


#-----------------------------------------------------------
# 3. Fill all missing values in all columns with nanplaceholder

# Check for values < -50 to see if nanplaceholder -99 is ok

# Select only numeric columns
numeric_df = df.select_dtypes(include=['number'])

# If there are no values less than -50 in any row of any column, proceed to fill NaNs with placeholders
if not (numeric_df < -50).any().any():
    nanplaceholder_num = -99
    nanplaceholder_str = str(nanplaceholder_num)

    # Handle categorical columns
    for col in df.select_dtypes(include=["category", "object"]).columns:
        # Add the placeholder to the category if it's not already present
        df[col] = df[col].cat.add_categories([nanplaceholder_str])
        # Fill NaNs with nanplaceholder_str
        df[col].fillna(nanplaceholder_str, inplace=True)

    # Handle numerical columns
    for col in df.select_dtypes(include='number').columns:
        # Fill NaNs with nanplaceholder_num
        df[col].fillna(nanplaceholder_num, inplace=True)
else:
    # Print warning message
    print("Warning: There are values less than -50 in the numeric columns. Cannot use -99 as NaN placeholder.")

    
# save again
df.to_parquet(parquet_nanplaceholder_path)
print(f"Dataframe NaNs have been filled with placeholder \"{nanplaceholder_num}\" and saved to {parquet_nanplaceholder_path}")


#-----------------------------------------------------------
# 4. Count number of identical occurences and write in new column, then remove dupes

# i) specify the columns to be grouped by, i.e the ones that have to be identical in order to be counted
# excluding "Date of Registration" and "ID" ==> they don't have to be identical 
group_cols = [col for col in df.columns if col != "Date of registration" and col != "ID"]

# ii) count number of members in the group, i..e. number of identical occurences and add as new column
df['#identical occurences'] = df.groupby(group_cols)['ID'].transform('count')  

print("Dataframe duplicate counts have been calculated.")

# iii) remove dupes
df = df.drop_duplicates(subset = group_cols, keep = "first")

# save again
df.to_parquet(parquet_dupecounts_reduced_path)
print(f"Duplicates have been dropped and dataframe saved to {parquet_dupecounts_reduced_path}")
#----------------------------------------------------------------------------------

print(df.info())
print(df.head())


Current working directory:  c:\Users\onepl\Desktop\Finanzen Behörden etc - Copy\Bewerbung\Fortbildungen\DS Bootcamp\Projekt CO2\Notebooks
Dataframe for year 2010-2015 has been read from CSV and datatypes specified:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2248123 entries, 0 to 2248122
Data columns (total 40 columns):
 #   Column                Dtype   
---  ------                -----   
 0   ID                    Int32   
 1   Country               category
 2   VFN                   category
 3   Mp                    category
 4   Mh                    category
 5   Man                   category
 6   MMS                   category
 7   Tan                   category
 8   T                     category
 9   Va                    category
 10  Ve                    category
 11  Mk                    category
 12  Cn                    category
 13  Ct                    category
 14  Cr                    category
 15  r                     Int32   
 16  m (kg)             

In [7]:
# in case needed to check intermediate output
df = pd.read_parquet(parquet_file_path)

In [8]:
# in case needed to check intermediate output
df = pd.read_parquet(parquet_nanplaceholder_path)

In [29]:
# in case needed to check intermediate output
df = pd.read_parquet(parquet_dupecounts_reduced_path)

In [13]:
# read first 1000 characters of csv to check content
with open(csv_file_path) as f: chars = f.read(1000)

In [14]:
print(chars)

ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,Ve,Mk,Cn,Ct,Cr,r,m (kg),Mt,Enedc (g/km),Ewltp (g/km),W (mm),At1 (mm),At2 (mm),Ft,Fm,ec (cm3),ep (KW),z (Wh/km),IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption ,ech,RLFI,Electric range (km)
121150000,FR,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,,e2*2001/116*0323*72,SR,DHE2,MT6UB390M0DG,DACIA,DUSTER,M1,M1,1,1337,1446,,126,,,,lpg,B,999,74,,e9 29 37,,1.7,,,P,2023,2023-03-27,7.8,Euro 6 AP,,
121150001,FR,IP-0093-JT1-1,SUBARU-SUZUKI-TOYOTA,TOYOTA,TOYOTA MOTOR EUROPE NV SA,,e6*2007/46*0322*05,XV7(EUM),AXVH71(E),AXVH71L-AEXDBW(2F),TOYOTA,TOYOTA CAMRY,M1,M1,1,1670,1782,,125,,,,petrol,H,2487,131,,e6 37,,0.8,,,P,2023,2023-07-21,5.5,Euro 6 AP,,
121150002,FR,IP-CX72_2020_00002-WF0-1,FORD,FORD WERKE GMBH,FORD-WERKE GMBH,,e13*2007/46*2387*06,LSK,C1LM2YX,5AAAZNAXVAA,FORD,MUSTANG MACH-E,M1,M1,1,2044,2187,,0,,,,electric,E,,221,172,,,,,,P,2023,2023-10-24,,AX,,440
121150003,FR,IP-HNS____ATN82444-VR3-0
